In [1]:
import hopsy
from PolyRound.api import PolyRoundApi
import os
import time
import numpy as np
import copy

In [2]:
model_path = os.path.join("hopsy","examples","test_data", "e_coli_core.xml")
polytope = PolyRoundApi.sbml_to_polytope(model_path)

Restricted license - for non-production use only - expires 2025-11-24


In [3]:
problem = hopsy.Problem(polytope.A, polytope.b)
problem = hopsy.add_box_constraints(problem, upper_bound=10_000, lower_bound=-10_000, simplify=True)
start = time.perf_counter()
problem = hopsy.round(problem)
print("Computing rounding transformation took", time.perf_counter()-start,"seconds")

Computing rounding transformation took 3.874317241999961 seconds


In [4]:
print(polytope.A.shape)
print(polytope.b.shape)
print(problem.A.shape)
print(problem.b.shape)

(190, 95)
(190,)
(190, 95)
(190,)


In [5]:
seed = 511
chains, rngs = hopsy.setup(problem, seed, n_chains=4)
n_samples = 10000
# Either use thinning rule, see  10.1371/journal.pcbi.1011378
# or use one-shot transformation (for expert users). We show one-shot transformation at the end.
thinning = int(1./6*problem.transformation.shape[1])

<h3> Sampling with Transform <h3>

In [28]:
%%script False
start = time.perf_counter()
accrate, samples = hopsy.sample(chains, rngs, n_samples, thinning=thinning, n_procs=4)
# accrate is 1 for uniform samples with the default chains given by hopsy.setup()
print("sampling with internal trafo took", time.perf_counter()-start,"seconds")
print(samples.shape)
rhat = np.max(hopsy.rhat(samples))
print("rhat:", rhat)
ess = np.min(hopsy.ess(samples)) / len(chains)
print("ess:", ess)

sampling with internal trafo took 1.3233584550000614 seconds
(4, 100000, 95)
rhat: 1.0001632483672327
ess: 15137.688482919848


<h3>Sampling without Transform<h3>

In [16]:
assert problem.transformation is not None
# deep copy enures that we do not edit the original problem
problem2 = copy.deepcopy(problem)
problem2.transformation=None
problem2.shift=None
seed = 512
chains, rngs = hopsy.setup(problem2, seed, n_chains=4)
n_samples = 100_000
# thinning is still advised when hard drive memory is limisted to not to store too many samples 
thinning = int(1./6*problem.A.shape[1])  

start = time.perf_counter()
accrate, sample_stack = hopsy.sample(chains, rngs, n_samples, thinning=thinning, n_procs=4)
# accrate is 1 for uniform samples with the default chains given by hopsy.setup()
print("sampling took", time.perf_counter()-start,"seconds")

print('sample shape', sample_stack.shape)
rhat = np.max(hopsy.rhat(sample_stack))
print("rhat:", rhat)
ess = np.min(hopsy.ess(sample_stack)) / len(chains)
print("ess:", ess)



sampling took 1.3485633060000737 seconds
sample shape (4, 100000, 95)
rhat: 1.0001668563228547
ess: 15199.932708453518


In [8]:
# Make copy of sample_stack
sample_stack_seq = sample_stack.copy()
sample_stack_pl = sample_stack.copy()

<h3>Transform Back: Sequential<h3>

In [9]:
# transform samples back all at once
shift_t = np.array([problem.shift]).T
start_trafo = time.perf_counter()
full_samples = np.zeros((len(chains), n_samples, sample_stack.shape[2]))
for i in range(len(chains)):
    full_samples[i] = (problem.transformation@sample_stack[i].T).T + np.tile(shift_t, (1, n_samples)).T
    
print("transformation took", time.perf_counter()-start_trafo,"seconds")
print('sample stats are the same (save numerics) before and after the linear transformation:')
rhat = np.max(hopsy.rhat(full_samples))
print("rhat:", rhat)
ess = np.min(hopsy.ess(full_samples)) / len(chains)
print("ess:", ess)

transformation took 0.09082596899997952 seconds
sample stats are the same (save numerics) before and after the linear transformation:
rhat: 1.0017171320252791
ess: 1429.2896264045694


<h2>Transform Back: Parallel<h2>

In [10]:
from numba import cuda
from numba import *
import numpy as np

In [114]:
# Device Properties to get execution config
# Get the current device
device = cuda.get_current_device()

# Access device properties
warp_size = device.WARP_SIZE
multi_processor_count = device.MULTIPROCESSOR_COUNT

print(warp_size)
print(multi_processor_count)
tpb = warp_size                     # threads per block
nb = multi_processor_count * 32     # number of blocks

32
16


In [34]:
print(shift_t.shape)
print(np.tile(shift_t, (1, n_samples)).shape)

(95, 1)
(95, 100000)


In [115]:
# This version needs A and b to be input as transposed.
"""
Perform Y = X.A + b
X : (n_samples, A.shape[0]), n_samples is parallelised
Y : same shape as X
b : Row vector
"""
@cuda.jit
def tranformv1(A:np.ndarray, b: np.ndarray, X: np.ndarray, Y: np.ndarray):
    idx = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(idx,X.shape[0],stride):
        for j in range(A.shape[1]):
            temp = 0
            for k in range(A.shape[0]):
                temp += X[i,k] * A[k,j]
            Y[i,j] = temp + b[0,j]

In [116]:
# Test Setup
N = 20
A = np.random.random((5,5)).astype(np.float32)
X = np.random.random((N,5)).astype(np.float32)
b = np.random.random((1,5)).astype(np.float32)
Y = np.zeros_like(X).astype(np.float32)

In [117]:
truth = X@A + np.tile(b,(N,1))
btruth = X@A + b

# tranformsl(A,b,X,Y)
dA = cuda.to_device(A)
dX = cuda.to_device(X)
db = cuda.to_device(b)
dY = cuda.to_device(Y)
tranformv1[nb,tpb](dA,db,dX,dY)
Y = dY.copy_to_host()


In [118]:
Y - truth

array([[ 0.0000000e+00, -1.1920929e-07,  0.0000000e+00, -1.1920929e-07,
         0.0000000e+00],
       [ 2.3841858e-07,  2.3841858e-07,  0.0000000e+00,  0.0000000e+00,
        -2.3841858e-07],
       [ 0.0000000e+00, -1.1920929e-07,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 2.3841858e-07, -2.3841858e-07,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00, -2.3841858e-07,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  2.3841858e-07,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00, -1.1920929e-07,  0.0000000e+00, -1.1920929e-07,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.1920929e-07,
         1.1920929e-07],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00, -1.192